In [1]:
# Import packages
import re
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [2]:
# Code Source: 
# https://gist.github.com/bbass444/273846cfe9bcefb4ec7d6c60b399935e

def rev_scrape_indeed(url):
    
    # Initialize dataframes
    out_df = pd.DataFrame()
    title_df = pd.DataFrame()
    reviewer_df = pd.DataFrame()
    location_df = pd.DataFrame()
    date_df = pd.DataFrame()
    rating_df = pd.DataFrame()
    review_df = pd.DataFrame()
    
    # Specify url:
    base_address = url
    
    # Look this part up on BS - understand but want more detail
    page1 = urlopen(base_address)
    soup1 = BeautifulSoup(page1, 'lxml')
    
    # Looking up total # of reviews from title html tag - \d+ finds all digits
    review_num = int(                      
        (str(re.findall('\d+', str(soup1.title).replace(",", ''))).replace("[", '').replace("]", '').replace("'", '')))
    
    # Iterate over reviews, 20 per page
    for i in range(0, review_num, 20): 
    #for i in range(0, 2): 
        print(i)  # Keeps track of progress
        sleep(2)  # Don't want ip to get blocked
        url = base_address + str(i)
        page = urlopen(url)
        soup = BeautifulSoup(page, 'lxml')
    
        # Title
        title_info = [] 
        title = soup.find_all(class_="cmp-review-title")
        for t in title:
            title_info.append(t.text)
        title_df = title_df.append(title_info, ignore_index=True)
        
        # Reviewer
        reviewer_info = [] 
        reviewer = soup.find_all(class_="cmp-reviewer-job-title")
        for r in reviewer:
            reviewer_info.append(r.text)
        reviewer_df = reviewer_df.append(reviewer_info, ignore_index=True)
        
        # Location
        location_info = [] 
        location = soup.find_all(class_="cmp-reviewer-job-location")
        for l in location:
            location_info.append(l.text)
        location_df = location_df.append(location_info, ignore_index=True)
        
        # Date
        date_info = [] 
        date = soup.find_all(class_="cmp-review-date-created")
        for d in date:
            date_info.append(d.text)
        date_df = date_df.append(date_info, ignore_index=True)
        
        # Rating
        rating_info = [] 
        rating = soup.find_all(class_="cmp-ratingNumber")
        for r in rating:
            rating_info.append(r.text)
        rating_df = rating_df.append(rating_info, ignore_index=True)
        
        # Review
        reviewbody = []  
        body = soup.find_all(itemprop="reviewBody")
        for b in body:
            reviewbody.append(b.text)  # appending each bs4 object above to empty list
        review_df = review_df.append(reviewbody, ignore_index=True)
        
        
        out_df = pd.concat([title_df, reviewer_df, location_df, date_df, rating_df, review_df], axis=1, ignore_index=True)
        out_df.columns = ['title', 'reviewer', 'location', 'date', 'rating', 'reviews']
    return out_df

In [3]:
google_reviews = rev_scrape_indeed('https://www.indeed.com/cmp/Google/reviews?start=')

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720


In [4]:
google_reviews.iloc[[2]]

,title,reviewer,location,date,rating,reviews
2,Poor management style at Google,Mechanical Design Engineer (Contract) (Former ...,"Mountain View, CA","January 24, 2019",1.0,Management at Google has very low regard for a...


In [5]:
tesla_reviews = rev_scrape_indeed('https://www.indeed.com/cmp/Tesla/reviews?start=')

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560


In [6]:
tesla_reviews.iloc[[2]]

,title,reviewer,location,date,rating,reviews
2,I love this job I want to go back,Brake Line Installer (Former Employee) –,"Fremont, CA","January 18, 2019",5.0,I love this job I would love to have another o...


In [7]:
# Code Source: 
# https://gist.github.com/bbass444/273846cfe9bcefb4ec7d6c60b399935e

# Remove featured review that is reapeated on every page
google_reviews = google_reviews.drop_duplicates(keep='first') 
tesla_reviews = tesla_reviews.drop_duplicates(keep='first')

In [8]:
# Write files to excel
# Code Source
# https://stackoverflow.com/questions/13437727/writing-to-an-excel-spreadsheet

google_reviews.to_excel('google_reviews.xlsx', sheet_name='sheet1', index=False)
tesla_reviews.to_excel('tesla_reviews.xlsx', sheet_name='sheet1', index=False)

# large date would need to use a different format, larger data use a db like mongo or sql
# mongo of nosql better handles unstructured data better